In [ ]:
%pwd

In [ ]:
from google.colab import drive
drive.mount('../content/drive', force_remount=True)  # 내가 경로 설정하는 듯

In [ ]:
cd ../content/drive/My Drive/30words_classification/

In [ ]:
%tensorflow_version 1.x

In [ ]:
import os
#from wav_tools_SV import *
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import librosa, librosa.display
from sklearn.model_selection import train_test_split

def minmax_Batch_size(X):
    minmax_X = []
    #print(X.shape.as_list())
    #exit()
    for i in range(len(X)):
        temp = X[i]
        scaler = MinMaxScaler()
        temp = scaler.fit_transform(temp)
        minmax_X.append(temp)

    return minmax_X

def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)

    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

############ 2. Feature Extraction ############
def plot_mfcc(mfcc, RECORD_FILE_NAME=None):
    plt.figure()
    librosa.display.specshow(mfcc, x_axis='time')  # y 축은 모르겠당 y_coords=13
    # plt.plot(mfcc[1], mfcc[0])
    plt.title('MFCC')
    plt.xlabel('Time [s]')
    plt.ylabel('MFCC Coefficeints')
    plt.colorbar()
    plt.show()

def match_1s(data):
    # 30sec 길이 통일
    num = 22050
    # print(data.shape, end=' ->')
    if len(data) < 22050:
        num = 22050 - len(data)
        temp = np.zeros(num) #* 1e-05
        data = np.append(data, temp)
    elif len(data) > 22050:
        data = data[:22050]

    #print(data.shape, end=' ')

    # (22050,) to column vector : (2250, 1)
    # data = data.reshape(len(data), 1)

    return data


def feature_mfcc(RECORD_FILE_NAME):

    # 조정할 수 있는 건 다 적어보자.

    # sr = 22050 = bitrate/2 -> Q. bitrate 와 어떤 관계?
    # Generate mfccs from a time series
    # t초당 sig.shape = (t*sr,)
    sig, sr = librosa.load(RECORD_FILE_NAME)  # , sr=sr
    # 만약, sr=16000, mfcc.shape = (n_mfcc,1251)
    #       sr=(default)22050, mfcc.shape = (n_mfcc, 1723)


    hop_length = 0
    if len(sig) == 22050:  # 128 -> mfcc Tx 301, 223 -> mfcc Tx 173
        hop_length = 128
    elif len(sig) == 38433:
        hop_length = 223  # Tx 173 으로 통일
    elif len(sig) < 22050:
        # print("smaller than", end=' ')
        sig = match_1s(sig)
        hop_length = 128

    else:
        # print(len(sig))
        sig = match_1s(sig)
        print("1s over")
        hop_length = 128

    n_mfcc = 24
    # n_mels = 20
    n_fft = 101
    fmin = 0
    fmax = None
    # sr = 16000


    mfcc = librosa.feature.mfcc(y=sig, sr=sr, hop_length=hop_length, fmin=fmin, fmax = fmax,
                                  n_fft= n_fft, n_mfcc=n_mfcc)
    # print("here", mfcc.shape)

    return mfcc

In [ ]:
X_train, X_test, Y_train, Y_test = np.load("./data.npy", allow_pickle=True)

In [ ]:
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# model
with tf.device('/gpu:0'):
    num_mfcc = 24
    num_class = 30
    Tx = 173

    X = tf.placeholder(tf.float32, [None, Tx, num_mfcc], name="X")
    Y = tf.placeholder(tf.float32, [None, num_class], name="Y")


    hidden_size = 32
    n_layers = 2
    learning_rate = 0.001
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")



    layers = [tf.nn.rnn_cell.GRUCell(num_units=hidden_size,activation='relu')
                            for layer in range(n_layers)]
    layers_drop = [tf.nn.rnn_cell.DropoutWrapper(
                        layer, state_keep_prob= keep_prob)
                        for layer in layers]

    multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(layers_drop)
    outputs, _state = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

    outputs = tf.contrib.layers.fully_connected(inputs=outputs, num_outputs=1  # 1로 줄이고..
                                                , activation_fn=None)  # tf.nn.relu 얘가 default 라니..

    # length of sequence to
    outputs_ = tf.reshape(outputs, shape=[-1, 1, Tx])
    W = tf.get_variable(name="last_W", shape=[1, Tx, num_class],
                        initializer= xavier_init(Tx, num_class))
    predict = tf.nn.conv1d(outputs_, filters=W, padding="VALID",)  # 1D 쉽다
    logits = tf.reshape(predict, shape=[-1, num_class], name="predict")

    hypothesis = tf.nn.softmax(logits, name="hypothesis")
    cost = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
    cost = tf.reduce_mean(cost, name='loss')

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, name="MyOpt")



In [ ]:
## traning

learning_rate = 0.001
training_epochs = 100
batch_size = 5
total_data = len(X_train)
test_acc = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver() # (max_to_keep=300)
    train_writer = tf.summary.FileWriter("./log/", sess.graph)

    # saver = tf.train.import_meta_graph('./models/GRU_2_layer/epoch19/cost_1.236314_data_num-26900.meta')
    # saver.restore(sess, tf.train.latest_checkpoint('./models/GRU_2_layer/epoch9/'))
    # graph = tf.get_default_graph()

    # X = graph.get_tensor_by_name("X:0")
    # Y = graph.get_tensor_by_name("Y:0")
    # cost = graph.get_tensor_by_name("loss:0")
    # cost = tf.reduce_mean(cost)
    # optimizer = graph.get_operation_by_name("MyOpt")
    # keep_prob = graph.get_tensor_by_name("keep_prob:0")
    # hypothesis = graph.get_tensor_by_name("hypothesis:0")


    for epoch in range(training_epochs):
        # min_cost = 1000
        epoch_path = './models/GRU_2_layer/epoch'+ str(epoch)
        os.makedirs(epoch_path, exist_ok = True)

        print("============ epoch", epoch, "============")
        for batch in range(0, total_data, batch_size):

            batch_xs = X_train[batch:batch + batch_size]
            batch_ys = Y_train[batch:batch + batch_size]
            minmax_batch_xs = minmax_Batch_size(batch_xs)

            feed_dict = {X: minmax_batch_xs , Y: batch_ys, keep_prob: 0.7}
            # batch 5개 cost_ 주의.
            cost_, _= sess.run([cost, optimizer], feed_dict=feed_dict)

            #train_writer.add_summary(summary, global_step= batch)
            if (batch % 100 ==0):
                print('epoch:', epoch,'data: %04d' % (batch), 'cost = %.9f'%(cost_))
                check_path = epoch_path + "/cost_%.6f_data_num"% (cost_)
                saver.save(sess, check_path, batch)

        print('Learning Finished!')

        minmax_X_test = minmax_Batch_size(X_test)

        minmax_X_test = minmax_X_test[0:3000]
        Y_test = Y_test[0:3000]

        print("label idx: ", end='')
        print(np.argmax(Y_test, 1))
        print("predict idx: ", end='')
        print(sess.run(tf.argmax(hypothesis, 1), feed_dict={X: minmax_X_test, keep_prob: 1}))

        ## 여러개일 때 유용
        correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        acc = sess.run(accuracy, feed_dict={X: minmax_X_test, Y: Y_test, keep_prob: 1})
        print("Accuracy: ", acc)
        test_acc.append(acc)    
        # np.save('./test_acc.npy', test_acc)
        print("test accurcy list")
        print(test_acc)
        
        if epoch > 5:
            if accuracy > test_acc[epoch-1] and test_acc[epoch-1] > test_acc[epoch-2] and test_acc[epoch-2] > test_acc[epoch-3]:
                print("Overfitting at epoch = ", epoch)
                exit()